In [15]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms

class MLP(nn.Module):
    def __init__(self,input,output):
        super().__init__()
        self.layer1 = nn.Linear(input, 16)
        self.layer2 = nn.Linear(16,20)
        self.layer3 = nn.Linear(20,output)

    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [16]:
import torch

class DataSet(torch.utils.data.Dataset):
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.data[idx], self.labels[idx]

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.genfromtxt('iris.data',delimiter=',')
data = np.delete(data,-1,axis=1)
classes = np.genfromtxt('iris.data',delimiter=',', dtype=str,usecols=[-1])
classes = np.unique(classes,return_inverse=1)[1]

data_treino, data_teste, classe_treino, classe_teste = train_test_split(data,classes,test_size=0.2,stratify=classes,random_state=123)

data_treino, data_validacao, classe_treino, classe_validacao = train_test_split(data_treino,classe_treino,test_size=0.1, stratify=classe_treino, random_state=1)

dataset = DataSet(data_treino,classe_treino)
data_train = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

dataset = DataSet(data_validacao,classe_validacao)
data_val = DataLoader(dataset, batch_size=len(data_validacao), shuffle=True, num_workers=0)

dataset = DataSet(data_teste,classe_teste)
data_test = DataLoader(dataset, batch_size=len(data_teste), shuffle=True, num_workers=0)

In [18]:
mlp = MLP(4,3)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [19]:
def train_one_epoch(NeuralNet,Loss,Optimizer,data_treino):
    NeuralNet.train(True)
    current_loss = 0.0
    total_samples = 0

    for data in data_treino:
        x,y = data
        x = x.to(torch.float)
        Optimizer.zero_grad()
        output = NeuralNet(x)
        loss = Loss(output,y)
        loss.backward()
        Optimizer.step()
        current_loss += loss.item()
        total_samples += len(x)

    return current_loss/total_samples

In [20]:
def validation_one_epoch(NeuralNet,Loss,val_data):
    NeuralNet.train(False)
    current_loss = 0.0
    total_samples = 0
    for data in val_data:
        x,y = data
        x = x.to(torch.float)
        output = NeuralNet(x)
        loss = Loss(output,y)
        current_loss += loss.item()
        total_samples += len(x)
    return current_loss/total_samples

In [21]:
epochs = 5000
train_loss_all_epoch = []
val_loss_all_epoch = []

mlp.train(True)
for epoch in range(epochs):
    print(f'Starting epoch {epoch+1}')

    train_loss_per_epoch = train_one_epoch(mlp,loss_function,optimizer,data_train)

    train_loss_all_epoch.append(train_loss_per_epoch)

    val_loss_per_epoch = validation_one_epoch(mlp,loss_function,data_train)
    
    val_loss_all_epoch.append(val_loss_per_epoch)
    
    print(f'Train Loss: {train_loss_per_epoch}')
    print(f'Val Loss: {val_loss_per_epoch}')

print('Training process has finished.')

Starting epoch 1
Train Loss: 0.04071248460699011
Val Loss: 0.040732082393434316
Starting epoch 2
Train Loss: 0.040802952316072255
Val Loss: 0.04055100017123752
Starting epoch 3
Train Loss: 0.04079006005216528
Val Loss: 0.040725151697794594
Starting epoch 4
Train Loss: 0.04044861925972833
Val Loss: 0.04049990464139868
Starting epoch 5
Train Loss: 0.04034193798347756
Val Loss: 0.04066880433647721
Starting epoch 6
Train Loss: 0.04075101569846824
Val Loss: 0.040469600094689265
Starting epoch 7
Train Loss: 0.0405384533935123
Val Loss: 0.04058253213211342
Starting epoch 8
Train Loss: 0.04050056250007064
Val Loss: 0.04064047336578369
Starting epoch 9
Train Loss: 0.04049771030743917
Val Loss: 0.040552514570730704
Starting epoch 10
Train Loss: 0.04062898181102894
Val Loss: 0.04053475349037736
Starting epoch 11
Train Loss: 0.040446188714769155
Val Loss: 0.04037903525211193
Starting epoch 12
Train Loss: 0.0403502760110078
Val Loss: 0.04038275943862067
Starting epoch 13
Train Loss: 0.0402951593752

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss_all_epoch)
plt.plot(val_loss_all_epoch)
plt.legend(['Treino', 'Validação'])
plt.xlabel('Épocas')

plt.show()

In [22]:
from sklearn.metrics import classification_report

mlp.train(False)

for data in data_test:
    x_test,y_test = data
    x_test = x_test.to(torch.float32)
    y_pred = mlp(x_test)
    y_pred = torch.Tensor.argmax(y_pred, dim=1)
    y_pred = torch.Tensor.numpy(y_pred)
    y_test = torch.Tensor.numpy(y_test)

print(classification_report(y_test,y_pred,zero_division=1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.80      0.89        10
           2       0.83      1.00      0.91        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

